# Compiling

Consider compiling a random n-qudit Clifford.

In [ ]:
from qdit.compiling import compile_circuit
from qdit.benchmarking import sample_clifford
from cirq import measure, LineQid

n_qudits = 2
dimension = 2

circuit = sample_clifford(n_qudits, dimension)
circuit.append(measure(LineQid(0, dimension=dimension), key="x"))
circuit.append(measure(LineQid(1, dimension=dimension), key="x"))

print(circuit)

0 (d=2): ───────CXc───F───S───CXc───F───Z───M('x')────────────────────
                │             │
1 (d=2): ───────CXt───S───────CXt───S───F───S────────F───Z───M('x')───


### Random Compiling
Qdit currently supports multi qudit random compiling which twirls single qubit gates to convert noise channels to Pauli depolarizing noise.

See "Noise tailoring for scalable quantum computation via randomized compiling" by Wallman and Emerson (2016).

The compiler class accessed by this compile_circuit function takes the parameter 'rc':

In [8]:
compiled = compile_circuit(circuit, d=dimension, rc=True)
print(compiled)

0 (d=2): ───────I────CXc───Z1───F───S───Z1───────CXc───I────────F───Z───M('x')────────────────────
                     │                           │
1 (d=2): ───────Z1───CXt───Z1───S───────W(1,1)───CXt───W(1,1)───S───F───S────────F───Z───M('x')───


### Measurement RC

Qdit also supports measurement randomized compiling, either alone...

In [9]:
compiled_mrc, measurement_corrections = compile_circuit(circuit, d=dimension, rc=False, mrc=True)
print(compiled_mrc)
print(measurement_corrections)

0 (d=2): ───────CXc───F───S───CXc───F───Z───────W(1,1)───M('x')────────────────
                │             │
1 (d=2): ───────CXt───S───────CXt───S───F───S───F────────Z────────I───M('x')───
{'x': [(0, 1)]}


... or together with RC:

In [10]:
compiled_rc_mrc, measurement_corrections = compile_circuit(compiled_mrc, d=dimension, rc=True, mrc=True)
print(compiled_rc_mrc)
print(measurement_corrections)

0 (d=2): ───────Z1───CXc───Z1───F───S───Z1───────CXc───I────────F───Z───W(1,1)───────W(1,1)───M('x')────────────────
                     │                           │
1 (d=2): ───────I────CXt───I────S───────W(1,1)───CXt───W(1,1)───S───F───S────────F───Z────────I────────I───M('x')───
{'x': [(0, 1)]}


### Native Gate Decomposition

Most importantly, Qdit's compiler handles decomposition of single qudit gates into native gates. Currently it only supports the following native gate sets:

* Rz 
* X90 *or* Ry
* CX, CZ (for any prime d), *or* iSWAP (for d=2 only)

In [ ]:
from cirq import unitary
from qdit.utils.math import unitary_equiv
u1 = unitary(circuit)
print("Circuit with Rz, Ry, and CZ")
compiled = compile_circuit(circuit, d=dimension, native=True, native_gates='RzRyCZ')
print(compiled)
u2 = unitary(compiled)

print("\nCircuit with X90, and iSWAP instead of Ry and CZ")
compiled = compile_circuit(circuit, d=dimension, native=True, native_gates='RzRxiSWAP')
print(compiled)
u3 = unitary(compiled)

print(unitary_equiv(u1,u2))

Circuit with Rz, Ry, and CZ
0 (d=2): ────────────────────────────CZc───Rz(1.00π)────Ry(0.50π)───Rz(-0.50π)───CZc───Rz(1.00π)────Ry(0.50π)───Rz(-1.00π)───M('')───
                                     │                                           │
1 (d=2): ───Rz(-1.00π)───Ry(0.50π)───CZt───Rz(-0.50π)───Ry(0.50π)───Rz(0.50π)────CZt───Rz(-0.50π)───M('')────────────────────────────

Circuit with X90, and iSWAP instead of Ry and CZ
0 (d=2): ───Rz(0.50π)────────────────────────────────────────────────────⨂───Rz(-0.50π)───X90───X90───X90───Rz(0.50π)───X90───Rz(2.00π)───Rz(0.50π)───⨂───Rz(-1.00π)───X90───X90───X90───Rz(0.50π)───X90───Rz(2.00π)───⨂───Rz(-0.50π)───X90───X90───X90───Rz(0.50π)───X90───Rz(2.00π)───Rz(0.50π)───⨂───Rz(1.00π)───X90───X90───X90───Rz(0.50π)───X90───Rz(2.00π)───Rz(-1.00π)───────────M('')───
                                                                         │                                                                            │                                  

In [ ]:
from cirq import Circuit, LineQid
# from qdit.gates import X90, Ry, Rz, CX, iSWAP, F, Sdg
# from qdit.compiling import compile_circuit
from qdit import X90, Ry, Rz, CX, iSWAP, F, Sdg, compile_circuit, unitary_equiv
import numpy as np
d=3
c = Circuit()
q0 = LineQid(0,d)
q1 = LineQid(1,d)
# Sdg(d)(control), H(d)(control), Sdg(d)(control)
c.append(Sdg(d)(q0))
c.append(F(d)(q0))
c.append(Sdg(d)(q0))
# c.append(X90()(q0))

# c.append(Rz(np.pi/2)(q0))

# c.append(Rz(np.pi/2)(q1))
# c.append(X90()(q1))
# c.append(Rz(3*np.pi/2)(q1))

# c.append(X90()(q1))

# c.append(iSWAP()(q0,q1))

# c.append(X90()(q0))

# c.append(iSWAP()(q0,q1))

# c.append(Rz(-np.pi/2)(q1))

print(c)
print(np.round(c.unitary()))

comp_c = compile_circuit(c, d, native=True, native_gates='RzRxiSWAP')
print(comp_c)
print(np.round(comp_c.unitary(),3))

print(unitary_equiv(c.unitary(), comp_c.unitary()))
# c = Circuit()
# c.append(CX(dimension=d)(q0,q1))
# comp_c = compile_circuit(c, d, native=True, native_gates='RzRxiSWAP')
# print(comp_c)
# print(c.unitary())

0 (d=3): ───Sdg───F───Sdg───
[[ 1.+0.j -0.+0.j -0.+0.j]
 [-0.+0.j  1.+0.j -0.+1.j]
 [-0.+0.j -0.+1.j  1.-0.j]]
0 (d=3): ───X9023───Rz(1.50π)23───X9023───Rz(2.00π)23───Rz(0.22π)23───Rz(0.94π)12───X9012───Rz(1.61π)12───X9012───Rz(2.00π)12───Rz(-0.06π)12───Rz(2.22π)23───X9023───Rz(1.50π)23───X9023───Rz(2.00π)23───
[[ 0.569-0.1j   -0.197+0.543j -0.197+0.543j]
 [-0.197+0.543j  0.569-0.1j   -0.197+0.543j]
 [-0.197+0.543j -0.197+0.543j  0.569-0.1j  ]]
True
